In [2]:
import redis
import pickle
import time
import uuid
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Dict, Optional

class MinuteDataPublisher:
    def __init__(self, config_path: str = "redis.conf"):
        """初始化发布器，完全参考RemoteSender的初始化逻辑"""
        self.redis_config = self._load_redis_config(config_path)
        self.redis = redis.Redis(
            host=self.redis_config["host"],
            port=self.redis_config["port"],
            password=self.redis_config["password"],
            decode_responses=False  # 保持二进制传输，与参考代码一致
        )
        self.task_queue = "function_calls"  # 与参考代码队列名称一致
        self._test_redis_connection()
        print(f"✅ 发布器初始化完成 | pandas版本: {pd.__version__}")

    def _load_redis_config(self, config_path: str) -> Dict[str, str]:
        """加载Redis配置，完全复用参考代码的解析逻辑"""
        config = {"host": "localhost", "port": 6379, "password": ""}
        try:
            with open(config_path, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if line.startswith("host="):
                        config["host"] = line.split("=", 1)[1].strip()
                    elif line.startswith("port="):
                        config["port"] = int(line.split("=", 1)[1].strip())
                    elif line.startswith("password="):
                        config["password"] = line.split("=", 1)[1].strip()
            return config
        except Exception as e:
            print(f"⚠️ 配置文件读取失败，使用默认配置: {e}")
            return config

    def _test_redis_connection(self):
        """测试Redis连接，参考RemoteSender的_test_connection方法"""
        try:
            self.redis.ping()
            print(f"✅ Redis连接成功 | {self.redis_config['host']}:{self.redis_config['port']}")
        except Exception as e:
            print(f"❌ Redis连接失败: {e}")
            raise SystemExit(1)

    def _get_valid_stocks(self, daily_data_path: str, target_date: Optional[str] = None) -> List[str]:
        """获取有效股票列表，优化：直接转换筛选条件格式，不修改原始df"""
        try:
            daily_df = pd.read_parquet(daily_data_path)
            # 不修改原始df的date类型，保持其原有格式（如datetime或字符串）
            
            # 基础筛选：未停牌股票（paused==0）
            filter_condition = (daily_df["paused"] == 0)
            
            # 关键优化：指定日期时，仅转换target_date格式与df.date匹配，不修改df
            if target_date:
                # 1. 获取df.date的原始类型
                if pd.api.types.is_datetime64_any_dtype(daily_df["date"]):
                    # 若df.date是datetime类型，将target_date转为datetime再匹配
                    target_date_dt = pd.to_datetime(target_date, format="%Y%m%d")
                    filter_condition &= (daily_df["date"].dt.strftime("%Y%m%d") == target_date)
                else:
                    # 若df.date是字符串类型，直接用target_date匹配（默认target_date是%Y%m%d格式）
                    filter_condition &= (daily_df["date"] == target_date)
            
            # 提取有效股票并去重
            valid_stocks = daily_df[filter_condition]["stock_code"].unique().tolist()
            print(f"📊 有效股票数量: {len(valid_stocks)} {'| 日期: ' + target_date if target_date else ''}")
            return valid_stocks
        except Exception as e:
            print(f"⚠️ 读取股票列表失败，使用默认列表: {e}")
            return []  # 默认股票

    def publish_tasks(
        self,
        func_name: str = "fetch_minute_stock_data",
        daily_data_path: str = r"D:\workspace\xiaoyao\data\stock_daily_price.parquet",
        batch_size: int = 50,  # 1. 改为批量50只股票
        max_queue_len: int = 10000,
        target_date: Optional[str] = None  # 2. 新增：支持指定日期
    ):
        """发布任务的核心方法，完全参考RemoteSender.call_remote_function的任务结构"""
        # 1. 准备基础数据
        # 3. 传入目标日期筛选股票
        valid_stocks = self._get_valid_stocks(daily_data_path, target_date)

        if not valid_stocks:
            print("❌ 无有效股票，退出发布")
            return

        # 2. 按股票批次发布任务
        total_batches = (len(valid_stocks) + batch_size - 1) // batch_size
        
        # 4. 如果指定了日期，则只抓取该日期
        if target_date:
            start_date = '20251029'
            end_date = target_date
        else:
            start_date = '20250101'
            end_date = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
        
        print(f"🚀 开始发布任务 | 总批次: {total_batches} | 日期范围: {start_date}-{end_date} | 每批股票数: {batch_size}")

        for batch_idx in range(total_batches):
            # 计算当前批次股票
            start = batch_idx * batch_size
            end = min(start + batch_size, len(valid_stocks))
            batch_stocks = valid_stocks[start:end]
            
            # 生成符合参考代码格式的任务ID
            task_id = f"task_{uuid.uuid4().hex[:8]}"
            
            # 控制队列长度，避免溢出
            while self.redis.llen(self.task_queue) >= max_queue_len:
                print(f"⏳ 队列已满（{max_queue_len}），等待30秒...")
                time.sleep(30)

            # 构建任务结构
            task = {
                "func_name": func_name,          
                "args": (start_date, end_date, batch_stocks),  
                "kwargs": {},                    
                "task_id": task_id               
            }

            # 序列化任务
            self.redis.rpush(self.task_queue, pickle.dumps(task))
            
            # 记录任务元信息
            self.redis.hset(
                "task_metadata",
                task_id,
                pickle.dumps({
                    "batch": batch_idx,
                    "stocks_count": len(batch_stocks),
                    "date_range": f"{start_date}-{end_date}"
                })
            )

            # 打印进度
            progress = (batch_idx + 1) / total_batches * 100
            print(f"📤 批次 {batch_idx+1}/{total_batches} ({progress:.1f}%) | 任务ID: {task_id[:8]}... | 股票数: {len(batch_stocks)}")

            # 5. 缩短发布间隔（从5秒改为1秒）
            if batch_idx < total_batches - 1:
                time.sleep(1)

        print(f"✅ 所有任务发布完成 | 总任务数: {total_batches} | 总股票数: {len(valid_stocks)}")

if __name__ == "__main__":
    try:
        publisher = MinuteDataPublisher(config_path="redis.conf")
        # 6. 可指定日期（如"20251015"），不指定则默认抓取历史到昨日
        # target_date = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")  # 默认为昨日
        # target_date = (datetime.now() - timedelta(days=0)).strftime("%Y%m%d")  # 默认为昨日
        target_date = "20251104"

        publisher.publish_tasks(
            func_name="fetch_minute_stock_data",
            daily_data_path=r"D:\workspace\xiaoyao\data\stock_daily_price.parquet",
            batch_size=600,  # 每批100只股票
            max_queue_len=1000,
            target_date=target_date  # 传入目标日期
        )
    except Exception as e:
        print(f"❌ 发布器执行失败: {e}")


✅ Redis连接成功 | 220.203.1.124:6379
✅ 发布器初始化完成 | pandas版本: 2.3.2
📊 有效股票数量: 5154 | 日期: 20251104
🚀 开始发布任务 | 总批次: 9 | 日期范围: 20251029-20251104 | 每批股票数: 600
📤 批次 1/9 (11.1%) | 任务ID: task_514... | 股票数: 600
📤 批次 2/9 (22.2%) | 任务ID: task_f46... | 股票数: 600
📤 批次 3/9 (33.3%) | 任务ID: task_126... | 股票数: 600
📤 批次 4/9 (44.4%) | 任务ID: task_acb... | 股票数: 600
📤 批次 5/9 (55.6%) | 任务ID: task_b4a... | 股票数: 600
📤 批次 6/9 (66.7%) | 任务ID: task_17d... | 股票数: 600
📤 批次 7/9 (77.8%) | 任务ID: task_eae... | 股票数: 600
📤 批次 8/9 (88.9%) | 任务ID: task_be8... | 股票数: 600
📤 批次 9/9 (100.0%) | 任务ID: task_e9b... | 股票数: 354
✅ 所有任务发布完成 | 总任务数: 9 | 总股票数: 5154


In [3]:
# import redis
# import pickle
# import time
# import uuid
# import pandas as pd
# from datetime import datetime, timedelta
# from typing import List, Dict, Optional

# class MinuteDataPublisher:
#     def __init__(self, config_path: str = "redis.conf"):
#         """初始化发布器，完全参考RemoteSender的初始化逻辑"""
#         self.redis_config = self._load_redis_config(config_path)
#         self.redis = redis.Redis(
#             host=self.redis_config["host"],
#             port=self.redis_config["port"],
#             password=self.redis_config["password"],
#             decode_responses=False  # 保持二进制传输，与参考代码一致
#         )
#         self.task_queue = "function_calls"  # 与参考代码队列名称一致
#         self._test_redis_connection()
#         print(f"✅ 发布器初始化完成 | pandas版本: {pd.__version__}")

#     def _load_redis_config(self, config_path: str) -> Dict[str, str]:
#         """加载Redis配置，完全复用参考代码的解析逻辑"""
#         config = {"host": "localhost", "port": 6379, "password": ""}
#         try:
#             with open(config_path, "r", encoding="utf-8") as f:
#                 for line in f:
#                     line = line.strip()
#                     if line.startswith("host="):
#                         config["host"] = line.split("=", 1)[1].strip()
#                     elif line.startswith("port="):
#                         config["port"] = int(line.split("=", 1)[1].strip())
#                     elif line.startswith("password="):
#                         config["password"] = line.split("=", 1)[1].strip()
#             return config
#         except Exception as e:
#             print(f"⚠️ 配置文件读取失败，使用默认配置: {e}")
#             return config

#     def _test_redis_connection(self):
#         """测试Redis连接，参考RemoteSender的_test_connection方法"""
#         try:
#             self.redis.ping()
#             print(f"✅ Redis连接成功 | {self.redis_config['host']}:{self.redis_config['port']}")
#         except Exception as e:
#             print(f"❌ Redis连接失败: {e}")
#             raise SystemExit(1)

#     def _get_valid_stocks(self, daily_data_path: str) -> List[str]:
#         """获取有效股票列表，参考日K数据筛选逻辑"""
#         try:
#             daily_df = pd.read_parquet(daily_data_path)
#             # 筛选未停牌股票（paused=0）并去重
#             valid_stocks = daily_df[daily_df["paused"] == 0]["stock_code"].unique().tolist()
#             print(f"📊 有效股票数量: {len(valid_stocks)}")
#             return valid_stocks
#         except Exception as e:
#             print(f"⚠️ 读取股票列表失败，使用默认列表: {e}")
#             return []  # 默认股票

#     def publish_tasks(
#         self,
#         func_name: str = "fetch_minute_stock_data",
#         daily_data_path: str = r"D:\workspace\xiaoyao\data\stock_daily_price.parquet",
#         batch_size: int = 1,
#         max_queue_len: int = 10000
#     ):
#         """发布任务的核心方法，完全参考RemoteSender.call_remote_function的任务结构"""
#         # 1. 准备基础数据

#         valid_stocks = self._get_valid_stocks(daily_data_path)
#         # valid_stocks = ['300770.XSHE', '300775.XSHE', '300778.XSHE', '300782.XSHE', '600103.XSHG', '601155.XSHG']

#         if not valid_stocks:
#             print("❌ 无有效股票，退出发布")
#             return

#         # 2. 按股票批次发布任务（每个任务包含完整日期范围）
#         total_batches = (len(valid_stocks) + batch_size - 1) // batch_size
#         start_date = '20250101'
#         # end_date 取昨日
#         end_date = (datetime.now() - timedelta(days=1)).strftime('%Y%m%d')
        
#         print(f"🚀 开始发布任务 | 总批次: {total_batches} | 日期范围: {start_date}-{end_date}")

#         for batch_idx in range(total_batches):
#             # 计算当前批次股票
#             start = batch_idx * batch_size
#             end = min(start + batch_size, len(valid_stocks))
#             batch_stocks = valid_stocks[start:end]
            
#             # 生成符合参考代码格式的任务ID
#             task_id = f"task_{uuid.uuid4().hex[:8]}"
            
#             # 控制队列长度，避免溢出（参考代码的流量控制逻辑）
#             while self.redis.llen(self.task_queue) >= max_queue_len:
#                 print(f"⏳ 队列已满（{max_queue_len}），等待30秒...")
#                 time.sleep(30)

#             # 构建任务结构：完全参考RemoteSender的task格式
#             task = {
#                 "func_name": func_name,          # 函数名
#                 "args": (start_date, end_date, batch_stocks),  # 位置参数（与函数参数对应）
#                 "kwargs": {},                    # 关键字参数（必须包含，即使为空）
#                 "task_id": task_id               # 任务ID
#             }

#             # 序列化任务：使用与参考代码相同的pickle.dumps
#             self.redis.rpush(self.task_queue, pickle.dumps(task))
            
#             # 记录任务元信息（便于结果处理）
#             self.redis.hset(
#                 "task_metadata",
#                 task_id,
#                 pickle.dumps({
#                     "batch": batch_idx,
#                     "stocks_count": len(batch_stocks),
#                     "date_range": f"{start_date}-{end_date}"
#                 })
#             )

#             # 打印进度（参考代码的简洁输出风格）
#             progress = (batch_idx + 1) / total_batches * 100
#             print(f"📤 批次 {batch_idx+1}/{total_batches} ({progress:.1f}%) | 任务ID: {task_id[:8]}...")

#             # 控制发布速度（低频任务特性）
#             if batch_idx < total_batches - 1:
#                 time.sleep(5)

#         print(f"✅ 所有任务发布完成 | 总任务数: {total_batches}")

# if __name__ == "__main__":
#     # 执行入口：与参考代码的main函数风格一致
#     try:
#         publisher = MinuteDataPublisher(config_path="redis.conf")
#         publisher.publish_tasks(
#             func_name="fetch_minute_stock_data",
#             daily_data_path=r"D:\workspace\xiaoyao\data\stock_daily_price.parquet",
#             batch_size=1,
#             max_queue_len=100
#         )
#     except Exception as e:
#         print(f"❌ 发布器执行失败: {e}")
